In [1]:
import Pkg
include("EVGONN.jl")
using .EVGONN
using RDatasets
using StatsBase

In [2]:
iris = dataset("datasets", "iris");
X = Array(iris[1:4]);
y = iris[5];
n = length(y);

In [3]:
dt = fit(ZScoreTransform, X, dims=1)
X = StatsBase.transform(dt, X)
X = reshape(X, (4, 1, n));

In [4]:
function get_indices(v)
    return CategoricalArrays.order(v.pool)[v.refs]
end
    
refs=get_indices(y);
y=convert(Vector{Int},refs);

In [5]:
y_mod = zeros(UInt8, (n, 3))
for i in 1:n
    y_mod[i, y[i]] = 0x01
end

In [12]:
nn = NeuralNetwork(4, (20,8), 3, 1, η=0.01, β1=0.01, β2=0.02);

In [13]:
function training(iters=10000)
    #nn.learning_rate = 0.1
    #nn.β1 = 0.1
    costs = zeros(n);
    foo(x) = min(0.0000000005x^2 + 0.01, 0.51)
    for it in 1:iters
        for i in 1:n
            result = train(prepare(X[:, 1, i]'), prepare(y_mod[i, :]'), nn)
            costs[i] = result["cost"]
        end
        #nn.learning_rate = foo(it)
        if it % 100 == 0
            println("error = ", sum(costs))
        end
    end
end

training (generic function with 2 methods)

In [14]:
training()

error = 45.15348324754067
error = 43.97765374342861
error = 43.354287502661066
error = 42.81951990933257
error = 42.26529105675342
error = 41.65147591564698
error = 40.95969677043291
error = 40.15878869304489
error = 39.22553021414053
error = 38.22186771360709
error = 37.240471065109226
error = 36.32870318374572
error = 35.4875364627363
error = 34.6963025680081
error = 33.938581780851536
error = 33.21092331009756
error = 32.51819471558205
error = 31.865227309323842
error = 31.25586136553954
error = 30.693033307566985
error = 30.177885523145093
error = 29.709492731707012
error = 29.285258272274174
error = 28.901476413900983
error = 28.55377802849098
error = 28.237424582691798
error = 27.947505258186403
error = 27.679073424665145
error = 27.427221666155877
error = 27.187117839313302
error = 26.954148346120117
error = 26.724202574139362
error = 26.49333290966668
error = 26.25678900737472
error = 26.010093360719686
error = 25.752624389403017
error = 25.48810292969524
error = 25.22012470636

In [9]:
function check()
    checks = Dict("HIT" => 0, "MISS" => 0)
    for i in 1:n
        #println(predict(prepare(X[:, 1, i]'), nn)["result"])
        #println(argmax(predict(prepare(X[:, 1, i]'), nn)["result"])[2])
        #println(y[i])
        if argmax(EVGONN.predict(prepare(X[:, 1, i]'), nn)["result"])[2] == y[i]
            checks["HIT"] += 1
        else
            checks["MISS"] += 1
        end
    end
    println("accuracy = $(checks["HIT"] / 150)%")
end

check (generic function with 1 method)

In [10]:
check()

accuracy = 0.92%
